<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/SI_logistico_sazonal_cont4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy.integrate import solve_ivp
from IPython.display import display

# Modelo SI com crescimento logístico, sazonalidade mista e mortalidade por doença
def si_logistic_model(t, y, beta_f, beta_d, mu, nu, K, c0, tpico, sigma, alpha):
    S, I = y
    N = S + I  # População total
    N = max(N, 1e-10)  # Evita divisão por zero

    # Função sazonal para a transmissão dependente da densidade (pico gaussiano)
    c_t = c0 * np.exp(-((t % 365 - tpico) ** 2) / (2 * sigma ** 2))

    # Equações diferenciais com transmissão mista
    dSdt = N * (nu - N*(nu-mu) / K) - (beta_f / N + beta_d * c_t) * S * I - mu * S
    dIdt = (beta_f / N + beta_d * c_t) * S * I - (mu + alpha) * I  # Transmissão mista

    return [dSdt, dIdt]

# Função para atualizar os gráficos dinamicamente
def update_plot(beta_f, beta_d, mu, nu, K, c0, tpico, sigma, alpha, N0, I0_percent, t_sim, n_points):
    I0 = N0 * (I0_percent / 100)  # Convertendo percentual para valor absoluto
    S0 = N0 - I0  # O restante será suscetível

    t_span = (0, t_sim * 365)  # Simulação em anos convertida para dias
    t_eval = np.linspace(0, t_sim * 365, n_points)  # Resolução no tempo controlada pelo slider

    # Resolver EDOs com RK45
    sol = solve_ivp(si_logistic_model, t_span, [S0, I0],
                    args=(beta_f, beta_d, mu, nu, K, c0, tpico, sigma, alpha),
                    method='RK45', t_eval=t_eval, rtol=1e-6, atol=1e-8)

    # Extraindo soluções
    t_values = sol.t
    S_values, I_values = sol.y
    N_values = S_values + I_values  # População total

    # Check for negative values and set to zero
    S_values[S_values < 0] = 0
    I_values[I_values < 0] = 0
    N_values = S_values + I_values  # Recalculate N to ensure consistency

    # Criar gráficos interativos
    fig, ax = plt.subplots(3, 1, figsize=(8, 12), dpi=100)

    # Gráfico N(t)
    ax[0].plot(t_values / 365, N_values, 'g-')
    ax[0].set_xlabel('t (anos)')
    ax[0].set_ylabel('N(t)')
    ax[0].grid()

    # Gráfico I(t)
    ax[1].plot(t_values / 365, I_values, 'r-')
    ax[1].set_xlabel('$t$ (anos)')
    ax[1].set_ylabel('$I(t)$')
    ax[1].grid()

    # Diagrama de fase I x N
    ax[2].plot(I_values, N_values, 'k-')
    ax[2].set_ylabel('$N(t)$')
    ax[2].set_xlabel('$I(t)$')
    ax[2].grid()

    plt.tight_layout()
    plt.show()

# Criando sliders para ajuste dos parâmetros
beta_f_slider = widgets.FloatSlider(value=0.07, min=0.0001, max=10, step=0.0001, description='beta_f', readout_format='.6f')
beta_d_slider = widgets.FloatSlider(value=0.0007, min=0.0001, max=10, step=0.0001, description='beta_d', readout_format='.6f')
mu_slider = widgets.FloatSlider(value=0.008, min=0.001, max=0.05, step=0.001, description='mu', readout_format='.6f')
nu_slider = widgets.FloatSlider(value=0.022, min=0.005, max=0.05, step=0.001, description='nu', readout_format='.6f')
K_slider = widgets.IntSlider(value=20000, min=500, max=100000, step=100, description='K')
c0_slider = widgets.FloatSlider(value=2.9, min=1, max=10, step=0.1, description='c0')
tpico_slider = widgets.IntSlider(value=211, min=0, max=365, step=1, description='t_pico')
sigma_slider = widgets.IntSlider(value=35, min=10, max=100, step=5, description='sigma')
alpha_slider = widgets.FloatSlider(value=0.01, min=0, max=5, step=0.001, description='alpha',readout_format='.6f')
N0_slider = widgets.IntSlider(value=10000, min=500, max=100000, step=100, description='N0 (pop inicial)')
I0_percent_slider = widgets.FloatSlider(value=27, min=0.1, max=50, step=0.1, description='I0 (%)')
t_sim_slider = widgets.IntSlider(value=3, min=1, max=30, step=1, description='Tempo (anos)')
n_points_slider = widgets.IntSlider(value=8000, min=1000, max=100000, step=100, description='N. Pontos', readout_format='d')

# Interface interativa
ui = widgets.VBox([beta_f_slider, beta_d_slider, mu_slider, nu_slider, K_slider, c0_slider,
                   tpico_slider, sigma_slider, alpha_slider, N0_slider, I0_percent_slider,
                   t_sim_slider, n_points_slider])
out = widgets.interactive_output(update_plot, {
    'beta_f': beta_f_slider, 'beta_d': beta_d_slider, 'mu': mu_slider, 'nu': nu_slider, 'K': K_slider,
    'c0': c0_slider, 'tpico': tpico_slider, 'sigma': sigma_slider, 'alpha': alpha_slider,
    'N0': N0_slider, 'I0_percent': I0_percent_slider, 't_sim': t_sim_slider, 'n_points': n_points_slider
})

display(ui, out)

Output()